In [1]:
import sys
sys.path.append('../')
sys.path.append('C:\\Users\\olegk\\Documents\\unifloc\\')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from datetime import date
from os import walk
import os

import uniflocpy.uTools.plotly_workflow as pw
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, plot

In [2]:
# TODO проверить первичную обработку данных путем построения графиков
# TODO разобраться с ошибками при первичной обработке данных
# TODO где среднее значение, а где протяжка
# TODO генерация входных данных для адаптации и восстановление отдельно (drop и resample, resample и interpolate)
# TODO резать данные кусками в связи с пропусками
# TODO сделать .py и работать с дебагом

In [3]:
def plot_corr_heatmap(df, filename_str):
    corrs = df.corr()
    figure = ff.create_annotated_heatmap(
       z=corrs.values,
       x=list(corrs.columns),
       y=list(corrs.index),
       annotation_text=corrs.round(2).values,
       showscale=True)
    figure.layout.update(
        margin=go.layout.Margin(
            l=400,
            r=50,
            b=100,
            t=250
        ))
    filename_str +='.html'
    plot(figure,filename=filename_str)
    
def initial_editing(data, wellname):
    if len(data.columns)==4:  
        del data[0]
    data.columns = [0, 1, 2]
    test_str = data[0][0]
    index = test_str.find(wellname)
    str_to_delete = test_str[:index] + wellname + '. '
    data[0] = data[0].str.replace(str_to_delete,"")
    data.index = data[1]
    del data[1]
    data.index = pd.to_datetime(data.index)
    return data

def extract_df_one_parametr_and_edit(data, list_of_params, number_of_param_in_list):
    extracted_df_one_param = data[data[0] == list_of_params[number_of_param_in_list]].copy()
    edited_df_one_param = extracted_df_one_param.rename(index=str, columns = {2: extracted_df_one_param[0][0]})
    del edited_df_one_param[0]
    return edited_df_one_param

def create_edited_df(data, parametrs_list):  # ускорить процесс
    init_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, 0)
    result = init_one_parametr_df
    for i in range(1, len(parametrs_list)):
        new_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, i)
        result = result.join([new_one_parametr_df], how = "outer", sort=True)
    return result

Получение списка скважин по названию соответствующих директорий

In [4]:
current_path = os.getcwd()
path_to_data = current_path + "\\data\\"
dirnames_list = []
for (dirpath, dirnames, filenames) in walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
dirnames_list

['1354',
 '1479',
 '1509',
 '1540',
 '1567',
 '1602',
 '1628',
 '202',
 '252',
 '326',
 '353',
 '507',
 '540',
 '569',
 '570',
 '601',
 '627',
 '658',
 '689',
 '693']

In [5]:
#files = dirnames_list
files = ['353']
test_file = None

Первичная обработка данных, разнесение параметров по столбцам

In [ ]:
for this_file_name in files:
    print("Первичная обработка данных со СУ для скважины " + this_file_name)
    
    this_file_path = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + ".csv"
    test_file = pd.read_csv(this_file_path,sep=';', header=None)
    if type(test_file.iloc[0][2]) == str:
        test_file = pd.read_csv(this_file_path,sep=';', header=None, skiprows = 1, low_memory=False)
    edited_file = initial_editing(test_file, this_file_name)
    big_list_parametrs = np.unique(edited_file[0])
    edited_file = create_edited_df(edited_file, big_list_parametrs)
    this_file_path_out = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + "_first_edit" +  ".csv"
    print("Сохранение обработанных данных в:")
    print(this_file_path_out)
    edited_file.to_csv(this_file_path_out)

Скважины с ошибками

In [63]:
this_file_name = 'Скв. 1354 (01.07.2018-31.03.2019)'

In [64]:
this_file_path = current_path + "\\data\\" + "1354" +  "\\" + this_file_name + ".xls"

In [65]:
this_file_path

'C:\\Users\\olegk\\Documents\\unifloc\\sandbox\\uTools\\data\\1354\\Скв. 1354 (01.07.2018-31.03.2019).xls'

In [69]:
test_file = pd.read_excel(this_file_path)
test_file.index = pd.to_datetime(test_file['Дата'], dayfirst = True, format = "%d.%m.%Y", infer_datetime_format=True)
del test_file['Дата']

In [70]:
test_file.head()

,Тип,Нсп,ТМ,Дисп,OIS,Qж ТМ,Qн ТМ,Обв ТМ,Рбуф ТМ,Рлин ТМ,...,R изол ТМ,R изоляц.,U раб.,Э/э (Wакт),Э/э реакт. (ТМ),Э/э СУ (ТМ),Э/э счет. (ТМ),U отп. (ТМ),УРЭ,УРЭ К
Дата,,,,,,,,,,,,,,,,,,,,,
2019-03-31,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,121.0,88.81,23.16,0,20.0,...,9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-30,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,120.0,88.95,19.33,0,20.1,...,9999.0,9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-29,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,120.0,88.70,21.77,0,20.2,...,9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-28,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,127.0,71.33,51.93,0,20.3,...,9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-27,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,127.0,92.92,21.64,0,20.3,...,9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
test_file = test_file[test_file.columns[5:]]

In [74]:
test_file = test_file.resample('3h').mean()
test_file = test_file.interpolate()

In [76]:
test_file.columns

Index(['Qж ТМ', 'Qн ТМ', 'Обв ТМ', 'Рбуф ТМ', 'Рлин ТМ', 'Рзатр', 'Рприем',
       'Загр. ПЭД (ТМ)', 'Нд', 'Темп. ПЭД', 'Тдвиг ТМ', 'F вращ ТМ', 'Dшт',
       'Рбуф', 'Рэцн ТМ', 'Pзаб(Pпр)', 'Qгаз ТМ', 'Cos ф (ТМ)', 'Пл. ж . ТМ ',
       'F ТМ', 'U AB (ТМ)', 'I', 'Tуст. (ТМ)', 'Tуст', 'T на приёме нас.',
       'Темп. ж. ТМ', 'Тнасос ТМ', 'I A ТМ', 'I C ТМ', 'I В ТМ', 'R изол ТМ',
       'R изоляц.', 'U раб.', 'Э/э (Wакт)', 'Э/э реакт. (ТМ)', 'Э/э СУ (ТМ)',
       'Э/э счет. (ТМ)', 'U отп. (ТМ)', 'УРЭ', 'УРЭ К'],
      dtype='object')

In [79]:
test_file['Рбуф']

Дата
2018-07-01 00:00:00    32.0
2018-07-01 03:00:00    32.0
2018-07-01 06:00:00    32.0
2018-07-01 09:00:00    32.0
2018-07-01 12:00:00    32.0
2018-07-01 15:00:00    32.0
2018-07-01 18:00:00    32.0
2018-07-01 21:00:00    32.0
2018-07-02 00:00:00    32.0
2018-07-02 03:00:00    32.0
2018-07-02 06:00:00    32.0
2018-07-02 09:00:00    32.0
2018-07-02 12:00:00    32.0
2018-07-02 15:00:00    32.0
2018-07-02 18:00:00    32.0
2018-07-02 21:00:00    32.0
2018-07-03 00:00:00    32.0
2018-07-03 03:00:00    32.0
2018-07-03 06:00:00    32.0
2018-07-03 09:00:00    32.0
2018-07-03 12:00:00    32.0
2018-07-03 15:00:00    32.0
2018-07-03 18:00:00    32.0
2018-07-03 21:00:00    32.0
2018-07-04 00:00:00    32.0
2018-07-04 03:00:00    32.0
2018-07-04 06:00:00    32.0
2018-07-04 09:00:00    32.0
2018-07-04 12:00:00    32.0
2018-07-04 15:00:00    32.0
                       ... 
2019-03-27 09:00:00    27.0
2019-03-27 12:00:00    27.0
2019-03-27 15:00:00    27.0
2019-03-27 18:00:00    27.0
2019-03-27 21:0

In [84]:
path_to_chess_out = current_path + "\\data\\" + "1354" +  "\\"  + "1354 " + 'шахматка обработанная' + ".csv"
print(path_to_chess_out)
test_file.to_csv(path_to_chess_out)

In [ ]:
test_file = pd.read_csv(this_file_path,sep=';', header=None)
if type(test_file.iloc[0][2]) == str:
    test_file = pd.read_csv(this_file_path,sep=';', header=None, skiprows = 1)

In [ ]:
test_file.head()

In [ ]:
edited_file = initial_editing(test_file, this_file_name)

In [ ]:
edited_file.head()

In [ ]:
big_list_parametrs = np.unique(edited_file[0])

In [ ]:
edited_file = create_edited_df(edited_file, big_list_parametrs)

In [ ]:
error_wells = ['689', '202', '353']

Построение графиков для всех скважин

In [ ]:
#for i in files:
for i in ['1509']:
    if i not in error_wells:
        print("Построение графиков для скважины " + i)
        well = i
        this_file_path = current_path + "\\data\\" + well +  "\\" + well + "_first_edit" +  ".csv"
        try:
            test = pd.read_csv(this_file_path, index_col = '1')
        except:
            test = pd.read_csv(this_file_path)
            test = test.rename(columns={"Unnamed: 0": "1"})
            test.index = test["1"]
        test.index = pd.to_datetime(test.index)
        test_resampled = test.resample('3h').mean()

        test_resampled = test_resampled.dropna(subset = ['Объемный дебит жидкости'])

        all_traces = pw.create_traces_list_for_all_columms(test_resampled, chosen_mode = 'lines+markers')

        data = all_traces
        path_to_plot = current_path + "\\data\\" + well +  "\\" + \
                    'Исходные данные (first_edit) по 3 часа без интерполяции для ' + well +  '.html'
        print("Сохранение графика в:")
        print(path_to_plot)
        pw.plot_subplots(data, path_to_plot, True)
        #plot_corr_heatmap(test_resampled, "Корреляционная карта для данных по 3 часа без интерполяции")

In [ ]:
#test = pd.read_csv("edited_kobzar2.csv", index_col = "2")
#test.index = pd.to_datetime(test.index)
#
#test_resampled = test.resample('3h').mean()
#
#test_resampled = test_resampled.dropna(subset = [' Объемный дебит жидкости'])
#
#all_traces = pw.create_traces_list_for_all_columms(test_resampled, chosen_mode = 'lines')
#
#data = all_traces
#pw.plot_subplots(data, 'Исходные данные по 3 часа без интерполяции' + '.html', True)
#